In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset('ms_marco', 'v1.1')

In [3]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 10047
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 82326
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 9650
    })
})

In [4]:
dataset = load_dataset('ms_marco', 'v1.1', split='train')

In [5]:
dataset

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 82326
})

In [6]:
def preprocess(example):
    positive_passages=[]
    for p in enumerate(example['passages']["is_selected"]):
        if p[1]==1:
            positive_passages.append(example["passages"]["passage_text"][p[0]])
    negative_passages = []
    for p in enumerate(example['passages']["is_selected"]):
        if p[1]==0:
            negative_passages.append(example["passages"]["passage_text"][p[0]])
    if (len(positive_passages)>0 and len(negative_passages)>=5):
        positive = positive_passages[0]
        negatives = negative_passages[:5]
        return {
            "query": example["query"],
            "positive": positive,
            "negatives": negatives
        }
    else:
        return {"query": None, "positive": None, "negatives": None}

In [7]:
processed_data = dataset.map(preprocess, remove_columns=dataset.column_names)

In [8]:
processed_data = processed_data.filter(lambda x: x['query'] is not None and x['positive'] is not None)

In [9]:
contrastive_pairs = []
for item in processed_data:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [10]:
len(contrastive_pairs)

74538

In [11]:
from torch.utils.data import DataLoader

In [12]:
class ContrastiveDataset:
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]
        return item["anchor"], item["positive"], item["negatives"]

In [13]:
contrastive_dataset = ContrastiveDataset(contrastive_pairs)

In [14]:
data_loader = DataLoader(contrastive_dataset, batch_size=32, shuffle=True)

In [15]:
len(data_loader)

2330

In [16]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [17]:
model = AutoModel.from_pretrained("bert-base-uncased")

In [18]:
from peft import LoraConfig, get_peft_model

In [19]:
lora_config = LoraConfig(
    task_type= "FEATURE_EXTRACTION" 
)

In [20]:
lora_model = get_peft_model(model, lora_config)

In [21]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 109,777,152 || trainable%: 0.2686


In [22]:
import torch

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
device

device(type='cuda')

In [25]:
import torch.optim as optim
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=5e-5)

In [26]:
import torch.nn.functional as F

In [27]:
def compute_laplacian(similarity_matrix):
    similarity_matrix = similarity_matrix.float()

    degree_matrix = torch.diag(torch.sum(similarity_matrix, dim=1))
    
    laplacian_matrix = degree_matrix - similarity_matrix

    return laplacian_matrix

In [28]:
def effective_distance(laplacian_matrix):
    laplacian_pseudo_inv = torch.pinverse(laplacian_matrix)
    effective_distances = []
    for anchor_idx in range(0, len(laplacian_matrix), 7): 
        anchor_node=torch.zeros(len(laplacian_matrix), dtype=torch.float32, device=device)
        anchor_node[anchor_idx] = 1

        distances = []
        for i in range(7-1):
            node=torch.zeros(len(laplacian_matrix), dtype=torch.float32, device=device)
            node[anchor_idx+(i+1)] = 1
            diff = anchor_node - node
            dist = torch.matmul(torch.matmul(diff.T, laplacian_pseudo_inv), diff)
            distances.append(dist)
        
        effective_distances.append(torch.stack(distances))
    return(effective_distances)

In [29]:
def effective_loss(distances):
    logits=-torch.stack(distances, dim=0)
    labels = torch.zeros(logits.size(0), dtype=torch.long, device=logits.device)
    loss = torch.nn.CrossEntropyLoss()(logits, labels)
    return loss

In [30]:
dataset_val = load_dataset('ms_marco', 'v1.1', split='validation')
processed_data_val = dataset_val.map(preprocess, remove_columns=dataset_val.column_names)
processed_data_val = processed_data_val.filter(lambda x: x['query'] is not None and x['positive'] is not None)
contrastive_pairs_val = []
for item in processed_data_val:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_val.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })
contrastive_dataset_val = ContrastiveDataset(contrastive_pairs_val)
data_loader_val = DataLoader(contrastive_dataset_val, batch_size=32, shuffle=True)
len(data_loader_val)

284

In [31]:
lora_model = lora_model.to(device)

In [32]:
def evaluate_mrr(model, data_loader_val):
    model.eval()

    total_rr = 0.0
    num_queries = 0

    with torch.no_grad():
        for batch in data_loader_val:
            anchor_text = batch[0]
            positive_text = batch[1]
            negative_texts = batch[2]

            anchor_input = tokenizer(anchor_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
            positive_input = tokenizer(positive_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

            anchor_embedding = model(**anchor_input).last_hidden_state[:, 0, :]
            positive_embedding = model(**positive_input).last_hidden_state[:, 0, :]
            negative_embedding = [model(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :] for neg in negative_texts]
            batch_embeddings = []
     
            for i in range(len(anchor_embedding)):
                sample_embeddings = torch.cat(
                    [anchor_embedding[i:i+1], 
                    positive_embedding[i:i+1],
                    torch.stack([neg[i] for neg in negative_embedding])],
                    dim=0
                )
                batch_embeddings.append(sample_embeddings)

            all_embeddings = torch.cat(batch_embeddings, dim=0)
            sim=F.cosine_similarity(all_embeddings.unsqueeze(1), all_embeddings.unsqueeze(0), dim=2)
            laplacian = compute_laplacian(sim)
            distances=effective_distance(laplacian)
            all_similarities=-torch.stack(distances, dim=0)

            sorted_similarities, sorted_indices = torch.sort(all_similarities, dim=1, descending=True)

            # Find the rank of the first relevant (positive) document
            positive_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1  # +1 to make rank 1-based
            total_rr += torch.sum(1.0 / positive_rank.float()).item()  # Reciprocal rank
            num_queries += len(positive_rank)

    mrr = total_rr / num_queries
    return mrr



In [33]:
import os
save_dir ="/dss/dsshome1/07/ra65bex2/srawat"
import time
epoch_metrics = []
num_epochs=3

In [34]:
for epoch in range(num_epochs):
    start_time = time.time()
    lora_model.train() 
    
    total_loss = 0.0
    for batch in data_loader:
      
        anchor_texts = batch[0]
        positive_texts = batch[1]
        negative_texts = batch[2]
        
        anchor_inputs = tokenizer(anchor_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        positive_inputs = tokenizer(positive_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    
        anchor_embedding = lora_model(**anchor_inputs).last_hidden_state[:, 0, :]
        positive_embedding = lora_model(**positive_inputs).last_hidden_state[:, 0, :]
        negative_embedding = [lora_model(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :] for neg in negative_texts]

        batch_embeddings = []
        for i in range(len(anchor_embedding)): 
            sample_embeddings = torch.cat(
                [anchor_embedding[i:i+1],  
                positive_embedding[i:i+1], 
                torch.stack([neg[i] for neg in negative_embedding])],
                dim=0
            )
            batch_embeddings.append(sample_embeddings)

        all_embeddings = torch.cat(batch_embeddings, dim=0)
        sim=F.cosine_similarity(all_embeddings.unsqueeze(1), all_embeddings.unsqueeze(0), dim=2)
        laplacian = compute_laplacian(sim)
        distances=effective_distance(laplacian)

        loss = effective_loss(distances)
        
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step() 
        
        total_loss += loss.item()
    save_path = os.path.join(save_dir, f"effective_checkpoint_epoch_{epoch+1}.pth")
    torch.save(lora_model, save_path)
    print(f"EPOCH {epoch+1}:")
    print(f"Checkpoint saved: {save_dir}")
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data_loader)}")
    mrr_validation = evaluate_mrr(lora_model, data_loader_val)
    #mrr_train = evaluate_mrr(lora_model, data_loader)
    #print(f"Mean Reciprocal Rank (MRR) for training set: {mrr_train:.4f}")
    print(f"Mean Reciprocal Rank (MRR) for validation set: {mrr_validation:.4f}")
    end_time = time.time()
    print(f"Epoch {epoch+1} took {(end_time - start_time) / 60:.4f} minutes.")
    print(f"\n")
    epoch_metrics.append({
    'epoch': epoch + 1,
    'training_loss': total_loss / len(data_loader),
    'mrr_validation': mrr_validation,
    'time_taken_minutes': (end_time - start_time) / 60
    })


/tmp/ipykernel_4031083/2546139923.py:16: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  dist = torch.matmul(torch.matmul(diff.T, laplacian_pseudo_inv), diff)


EPOCH 1:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat
Epoch 1/3, Loss: 1.7918644404206665


Mean Reciprocal Rank (MRR) for validation set: 0.3944
Epoch 1 took 35.9377 minutes.




EPOCH 2:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat
Epoch 2/3, Loss: 1.7918178133186864


Mean Reciprocal Rank (MRR) for validation set: 0.3953
Epoch 2 took 35.8951 minutes.




EPOCH 3:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat
Epoch 3/3, Loss: 1.7918393819628866


Mean Reciprocal Rank (MRR) for validation set: 0.3941
Epoch 3 took 35.8920 minutes.




In [35]:
import json
with open(save_dir + '/effective_epoch_metrics.json', 'w') as f:
    json.dump(epoch_metrics, f)